In [1]:
# OG DATASET --- Build WIDE Energy table: ObjectId; Country; Continent; <TechUnit>__FYYYY for all years ---

import re
import pandas as pd

# 1) Load + ISO2 filter
csv_path = "Renewable_Energy.csv"
f = pd.read_csv(csv_path)
iso2_clean = f["ISO2"].astype(str).str.strip()
f = f[iso2_clean.ne("") & f["ISO2"].notna()].copy()

# 2) Detect FYYYY year columns
year_cols = sorted([c for c in f.columns if re.fullmatch(r"F\d{4}", c)])
for c in year_cols:
    f[c] = pd.to_numeric(f[c], errors="coerce")

# 3) Standardize Technology + extract MW/GWh
tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
f["Technology_std"] = f["Technology"].map(tech_map).fillna(f["Technology"])

def extract_abbr(unit: str):
    if not isinstance(unit, str):
        return None
    m = re.search(r"\b(MW|GWh)\b", unit, flags=re.IGNORECASE)
    return m.group(1).upper() if m else None

f["UnitAbbr"] = f["Unit"].apply(extract_abbr)
f = f[f["UnitAbbr"].notna()].copy()

# 4) TechUnit = "<Tech> (<MW|GWh>)"
f["TechUnit"] = f["Technology_std"].astype(str).str.strip() + " (" + f["UnitAbbr"] + ")"

# 5) Continent from ISO2 using libraries (comprehensive)

def add_continent_from_iso2(df: pd.DataFrame) -> pd.DataFrame:
    s = df["ISO2"].astype(str).str.strip().str.upper().replace({"": None, "NA": None, "NAN": None})

    # Preferred: country_converter
    try:
        import country_converter as coco
        cc = coco.CountryConverter()
        # coco returns a list; convert back to a Series aligned to index
        cont_list = cc.convert(names=s.tolist(), src="ISO2", to="continent", not_found=None)
        cont_series = pd.Series(cont_list, index=df.index)
        df["Continent"] = cont_series.fillna("Unknown")
        return df
    except Exception:
        pass

    # Fallback: pycountry_convert
    try:
        from pycountry_convert import (
            country_alpha2_to_continent_code as a2_to_cc,
            convert_continent_code_to_continent_name as cc_to_name,
        )
        def conv(a2):
            try:
                return cc_to_name(a2_to_cc(a2))
            except Exception:
                return None
        cont_series = s.map(conv).fillna("Unknown")
        df["Continent"] = cont_series
        return df
    except Exception:
        # Last resort: mark unknown (so pipeline doesn’t break)
        df["Continent"] = "Unknown"
        return df

# Apply
f = add_continent_from_iso2(f)

# 6) Ensure ObjectId exists (if not in CSV, synthesize)
if "ObjectId" not in f.columns:
    f["ObjectId"] = range(1, len(f) + 1)

# 7) Build WIDE table: index = (ObjectId, Country, Continent); columns = "<TechUnit>__FYYYY"
#   a) aggregate per country+techunit (sum over duplicates)
agg = f.groupby(["Country","Continent","TechUnit"], as_index=False)[year_cols].sum()    #! DEBUG REMOVED: "ObjectId",
#   b) long -> add combined column name "<TechUnit>__FYYYY"
long = agg.melt(id_vars=["Country","Continent","TechUnit"],                             #! DEBUG REMOVED: "ObjectId",
                value_vars=year_cols, var_name="YearCol", value_name="Value")
long["col"] = long["TechUnit"] + "__" + long["YearCol"]
#   c) wide pivot
wide = (long.pivot_table(index=["Country","Continent"], columns="col",                  #! DEBUG REMOVED: "ObjectId",
                         values="Value", aggfunc="sum")
            .reset_index()
            .fillna(0.0))

# 8) Output objects you’ll use later
df_wide = wide  # columns: ObjectId, Country, Continent, <TechUnit>__FYYYY, ...
tech_options = sorted(f["TechUnit"].unique().tolist())  # all TechUnits (MW+GWh)
xy_kwargs = {f"XY_var{i+1}": t for i, t in enumerate(tech_options)}  # for widget call

print(f"Wide shape: {df_wide.shape}  | TechUnits: {len(tech_options)}  | Years: {len(year_cols)}")

# Consistent color palette for continents
continent_colors = {
    "Asia": "#1f77b4",
    "Europe": "#ff7f0e",
    "Africa": "#2ca02c",
    "Oceania": "#d62728",
    "America": "#9467bd",
}

df_wide

Wide shape: (219, 232)  | TechUnits: 10  | Years: 23


col,Country,Continent,Bio (GWH)__F2000,Bio (GWH)__F2001,Bio (GWH)__F2002,Bio (GWH)__F2003,Bio (GWH)__F2004,Bio (GWH)__F2005,Bio (GWH)__F2006,Bio (GWH)__F2007,...,Wind (MW)__F2013,Wind (MW)__F2014,Wind (MW)__F2015,Wind (MW)__F2016,Wind (MW)__F2017,Wind (MW)__F2018,Wind (MW)__F2019,Wind (MW)__F2020,Wind (MW)__F2021,Wind (MW)__F2022
0,"Afghanistan, Islamic Rep. of",Asia,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.1,0.1,0.1,0.1,0.4,0.4,0.40,0.4,0.4,0.4
1,Albania,Europe,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
2,Algeria,Africa,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,10.2,10.2,10.2,10.2,10.0,10.00,10.0,10.0,10.0
3,American Samoa,Oceania,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,"Andorra, Principality of",Europe,0.000,0.000,0.000,0.000,1.000,1.000,4.000,19.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,Vietnam,Asia,300.886,293.138,342.400,337.094,312.986,298.974,334.390,347.934,...,53.0,53.0,136.0,160.2,204.7,236.7,374.55,518.0,4118.0,5065.0
215,West Bank and Gaza,Asia,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
216,"Yemen, Rep. of",Asia,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
217,Zambia,Africa,39.013,39.013,39.013,39.013,39.013,39.366,39.013,38.660,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


In [2]:
from Isea.scatter import ScatterBrush
import re

# Year range
years_int = sorted({int(m.group(1)) for c in df_wide.columns
                    for m in [re.search(r"__F(\d{4})$", str(c))] if m})
year_min, year_max = years_int[0], years_int[-1]


xy_kwargs = {f"XY_var{i+1}": t for i, t in enumerate(tech_options, start=1)}

rows = df_wide.to_dict("records")   # full table; widget remaps per year

w_scatter = ScatterBrush(
    data=rows,
    x=tech_options[1], y=tech_options[3],
    color="Continent",
    colorMap=continent_colors,
    label="Country",      # ✅ display name
    key="Country",        # ✅ stable unique id (one row per country now)
    width=1200, height=500,
    panel_position="right", panel_width=320, panel_height=220,
    YearMin=year_min, YearMax=year_max,
    **xy_kwargs,
)
display(w_scatter)

ScatterBrush(data=[{'Country': 'Afghanistan, Islamic Rep. of', 'Continent': 'Asia', 'Bio (GWH)__F2000': 0.0, '…

In [3]:
# --- Minimal linked view: second scatter shows only the selected countries ---

import pandas as pd
from Isea.scatter import ScatterBrush

# Pull current axis/fields from the first widget (robust to either attr or .options)
opts = getattr(w_scatter, "options", {}) or {}
x_col = getattr(w_scatter, "x", None) or opts.get("x")
y_col = getattr(w_scatter, "y", None) or opts.get("y")
label_col = getattr(w_scatter, "label", None) or opts.get("label") or "Country"
color_col = getattr(w_scatter, "color", None) or opts.get("color") or "Continent"
key_col   = getattr(w_scatter, "key", None)   or opts.get("key")   or "Country"


# print(opts, x_col, y_col, label_col, color_col, key_col)
# XY_var* and year range from earlier cells
xy_kwargs = {f"XY_var{i+1}": t for i, t in enumerate(tech_options, start=1)}

# Empty second widget (data will be filled on first selection)
w_scatter_sel = ScatterBrush(
    data=pd.DataFrame([], columns=[label_col, color_col, key_col]).to_dict("records"),
    x=x_col, y=y_col,
    color=color_col,colorMap=continent_colors, 
    label=label_col, key=key_col,
    width=900, height=450, panel_position="right", panel_height=160,
    # title="Linked view (selection from first scatter)",
    YearMin=year_min, YearMax=year_max,
    **xy_kwargs,
)
display(w_scatter_sel)

# Keep a df for downstream cells
df_selected = pd.DataFrame(columns=df_wide.columns)


def _link_selection_to_second(change):
    global df_selected
    sel = change.get("new") or {}
    rows = sel.get("rows") or []

    # countries from the selection (row has Country or fallback to label/key)
    countries = [r.get("Country") or r.get(label_col) or r.get(key_col) for r in rows]
    countries = [c for c in countries if isinstance(c, str)]

    # Update df_selected (full wide rows so year/XY remain interactive)
    if countries:
        df_selected = df_wide[df_wide["Country"].isin(countries)].copy()
    else:
        df_selected = pd.DataFrame(columns=df_wide.columns)

    # Keep linked widget in sync:
    #  - data: full rows (so its slider & XY pills work)
    #  - axes: mirror current axes from the first widget
    cur_x = getattr(w_scatter, "x", None) or opts.get("x")
    cur_y = getattr(w_scatter, "y", None) or opts.get("y")
    w_scatter_sel.data = (
        df_selected.assign(key=lambda d: d["Country"], label=lambda d: d["Country"])
                   .to_dict("records")
    )
    if cur_x and cur_y:
        w_scatter_sel.x = cur_x
        w_scatter_sel.y = cur_y

    # optional: clear selection inside the second chart each update
    w_scatter_sel.selection = {"type": None, "keys": [], "rows": [], "epoch": int(__import__("time").time()*1000)}

# Wire first -> second (live updates)
w_scatter.observe(_link_selection_to_second, names="selection")

C:\Users\20193915\AppData\Local\Temp\ipykernel_5808\3331563500.py:21: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  data=pd.DataFrame([], columns=[label_col, color_col, key_col]).to_dict("records"),


ScatterBrush(options={'x': 'Bio (MW)', 'y': 'Fossil (MW)', 'label': 'Country', 'color': 'Continent', 'key': 'C…

In [29]:
# ---- Minimal EV scatter prep (Region×Mode; exact column names) ----
import pandas as pd
import re
from Isea.scatter import ScatterBrush

df = pd.read_csv("Global_EV_clean.csv")

# Helper: pivot (region,mode,year) → (region,mode) with <name>__FYYYY columns
def pivot_param(data, parameter, powertrain=None, name="Val"):
    q = data[data["parameter"] == parameter]
    if powertrain is not None:
        q = q[q["powertrain"] == powertrain]
    q = q[q["mode"] != "EV"]                                 # drop generic mode
    if q.empty:
        return pd.DataFrame(columns=["region","mode"])
    g = q.groupby(["region","mode","year"], as_index=False)["value"].sum()
    p = g.pivot(index=["region","mode"], columns="year", values="value").sort_index(axis=1)
    p.columns = [f"{name}__F{int(y)}" for y in p.columns]
    return p.reset_index()

# ChargingStations: sum fast+slow per region-year, replicate to all non-EV modes, then pivot
cp = df[(df["parameter"] == "EV charging points") & (df["powertrain"].isin(["fast","slow"]))]
rgy = (cp.groupby(["region","year"], as_index=False)["value"].sum()
         .rename(columns={"value":"ChargingStations"}))
rgy["year"] = rgy["year"].astype(int)
base_modes = df.loc[df["mode"] != "EV", ["region","mode"]].drop_duplicates()
rep = base_modes.merge(rgy, on="region", how="left")
cs = (rep.pivot_table(index=["region","mode"], columns="year", values="ChargingStations", aggfunc="first")
         .sort_index(axis=1)
         .reset_index())
cs.columns = [f"ChargingStations__F{int(c)}" if c not in ("region","mode") else c for c in cs.columns]

# Metrics for X/Y
cfg = {
    "StockBEV":   ("EV stock",       "BEV"),
    "StockFCEV":  ("EV stock",       "FCEV"),
    "StockPHEV":  ("EV stock",       "PHEV"),
    "SalesBEV":   ("EV sales",       "BEV"),
    "SalesFCEV":  ("EV sales",       "FCEV"),
    "SalesPHEV":  ("EV sales",       "PHEV"),
    "SalesShare": ("EV sales share", "EV"),
    "StockShare": ("EV stock share", "EV"),
}
blocks = [pivot_param(df, p, pt, name=k) for k,(p,pt) in cfg.items()] + [cs]

# Merge on (region, mode)
wide = base_modes.copy()
for b in blocks:
    if not b.empty:
        wide = wide.merge(b, on=["region","mode"], how="outer")

# Year range + initialize bare columns for latest year (widget expects this)
yrs = sorted({int(m.group(1)) for c in wide.columns for m in [re.search(r"__F(\d{4})$", str(c))] if m})
yearMin, yearMax = (min(yrs), max(yrs)) if yrs else (None, None)

xyVars = list(cfg.keys()) + ["ChargingStations"]
if yearMax is not None:
    for v in xyVars:
        col = f"{v}__F{yearMax}"
        wide[v] = pd.to_numeric(wide[col], errors="coerce").fillna(0.0) if col in wide.columns else 0.0

# Minimal id/label; color by mode
wide["id"] = wide["region"] + "|" + wide["mode"]
wide["label"] = wide["region"] + " • " + wide["mode"]

# Scatter
w = ScatterBrush(
    wide,
    x="ChargingStations",   # defaults; switch in UI
    y="SalesBEV",
    key="id",
    label="label",
    color="mode",
    legend=True,
    legendPosition="right",
    xyVars=xyVars,
    yearMin=yearMin,
    yearMax=yearMax,
    width=1080,
    height=520,
    panel_position="right",
    panel_width=340,
)
w


ScatterBrush(data=[{'region': 'Australia', 'mode': 'Cars', 'StockBEV__F2010': None, 'StockBEV__F2011': 49.0, '…

In [33]:
# --- Minimal linked view: second scatter mirrors selection + axes (options-based) ---
from Isea.scatter import ScatterBrush

def _get_axes(widget):
    opts = getattr(widget, "options", {}) or {}
    return opts.get("x"), opts.get("y")

x0, y0 = _get_axes(w)

# empty second widget; same config as the first
w_link = ScatterBrush(
    wide.iloc[0:0].to_dict("records"),
    x=x0, y=y0,
    key="id", label="label", color="mode",
    legend=True, legendPosition="right",
    xyVars=xyVars, yearMin=yearMin, yearMax=yearMax,
    width=1080, height=420, panel_position="right", panel_width=340,
)
display(w_link)

def _sync(*_):
    # current axes from the first widget
    x, y = _get_axes(w)
    # current selection
    sel = w.selection if isinstance(w.selection, dict) else {}
    keys = sel.get("keys", []) or []
    sub = wide[wide["id"].isin(keys)] if keys else wide.iloc[0:0]

    # push data
    w_link.data = sub.to_dict("records")
    # push axes via options (keep other options intact)
    w_link.options = {**(w_link.options or {}), "x": x, "y": y}
    # optional: clear selection inside the second chart each update
    w_link.selection = {"type": None, "keys": [], "rows": [], "epoch": 0}

# react to selection and option (axis) changes
w.observe(lambda ch: _sync(), names="selection")
w.observe(lambda ch: _sync(), names="options")

# initial paint
_sync()


ScatterBrush(options={'x': 'ChargingStations', 'y': 'SalesBEV', 'label': 'label', 'color': 'mode', 'key': 'id'…

c:\Users\20193915\OneDrive - TU Eindhoven\UTEC - Data visualization\Proyect-Visualization\venv\Lib\site-packages\jupyter_client\session.py:721: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)
